Explicaciones para NLP utilizando SHAP
======================================

Introducción
------------

TODO

Para una introducción más detallada puede ver la entrada del blog: [Model interpretability — Making your model confesses: SHAP](https://santiagof.medium.com/model-interpretability-making-your-model-confess-shapley-values-5fb95a10a624)

¿Como funciona?
---------------

TODO

Para mas información sobre esta librería visite: https://shap.readthedocs.io/en/latest/index.html

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

In [16]:
!pip install transformers --quiet
!pip install shap --quiet

Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [ ]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip -qq tweet_classification_bert.zip

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Cargamos el conjunto de datos con el que se entrenó el modelo en caso de necesitarlo

In [ ]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Cargando un modelo de NLP
-------------------------

Cargaremos el modelo que fue descargado anteriornmente utilizando la librería de `transformers`. Note que cargamos tanto el `tokenizer` como el modelo propiamente dicho.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "tweet_classification_bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Recordemos que nuestro modelo predice los sectores a los que pertenecería el tweet, siendo ellos:

In [ ]:
target_names = ['ALIMENTACION', 'AUTOMOCION', 'BANCA', 'BEBIDAS', 'DEPORTES', 'RETAIL', 'TELCO']

Generando explicaciones con SHAP
--------------------------------

In [ ]:
sample = ["Nos estafaron en carrefour. No vuelvo a comprar alli jamas"]

In [ ]:
import shap

explainer = shap.Explainer(model, tokenizer)
shap_values = explainer(sample)

### Visualizando las explicaciones

In [ ]:
shap.plots.text(shap_values)

Podemos analizar el impacto de una sola clase:

In [ ]:
shap.plots.text(shap_values[:, :, "ALIMENTACION"])

Visualizando las palabras que tienen el mayor impacto en una clase determinada:

In [ ]:
shap.plots.bar(shap_values[:,:,"ALIMENTACION"].mean(0), order=shap.Explanation.argsort)

En los ejemplos anteriores, explicamos la salida directa del objeto pipline, que son las probabilidades de clase. A veces tiene más sentido trabajar en un espacio de probabilidades logarítmicas donde es natural sumar y restar efectos (la suma y la resta corresponden a la suma o resta de bits de información de evidencia).

In [ ]:
logit_explainer = shap.Explainer(shap.models.TransformersPipeline(pred, rescale_to_logits=True))

logit_shap_values = logit_explainer(sample)
shap.plots.text(logit_shap_values)

### Analizando las predicciones

In [ ]:
x = [
    "Nos estafaron en carrefour",
    "Nos estafaron en el super",
    "Nos estafaron en la gondola",
]

In [ ]:
y = [
    0,
    0,
    0
]

Utilizando un Teacher Forcing y creamos un Text masker

In [ ]:
teacher_forcing_model = shap.models.TeacherForcing(model, tokenizer)
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)

Creamos el explicador:

In [ ]:
explainer = shap.Explainer(teacher_forcing_model, masker)

Generamos las explicaciones:

shap_values = explainer(x, y)

Ahora que hemos generado los valores SHAP, podemos echar un vistazo a la contribución de los tokens en la entrada que impulsa a generar una predicción en la clase alimentación. Nota: El color rojo indica una contribución positiva mientras que el color azul indica una contribución negativa y la intensidad del color muestra su fuerza en la dirección respectiva.

In [ ]:
shap.plots.text(shap_values)